In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Código para aplicar la técnica Spectrogram EMDA Augmenter y guardar los espectrogramas

# Instalar las dependencias
!pip install librosa matplotlib

# Importar librerías
import librosa
import numpy as np
import os
import matplotlib.pyplot as plt


# Configuración de rutas
input_folder = '/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/DATA'  # Carpeta con audios originales
output_folder = '/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/RESULTS/SpectrogramAugmentation/EMDAAugmenter/SPECTROGRAMS'  # Carpeta para guardar los espectrogramas modificados
os.makedirs(output_folder, exist_ok=True)

# Función para aplicar EMDA Augmentation
def apply_emda(audio1, audio2, alpha, beta, delay, sr):
    # Convertir a espectrogramas
    spec1 = librosa.stft(audio1)
    spec2 = librosa.stft(audio2)

    # Aplicar retraso temporal a uno de los espectrogramas
    delay_samples = int(delay * sr / 512)  # Ajustar por el tamaño de la ventana
    spec2_shifted = np.roll(spec2, shift=delay_samples, axis=1)

    # Interpolación ponderada de los espectrogramas
    combined_spec = alpha * spec1 + (1 - alpha) * spec2_shifted
    return combined_spec

# Función para guardar el espectrograma como imagen
def save_spectrogram(spectrogram, sr, file_name):
    plt.figure(figsize=(10, 6))
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(spectrogram), ref=np.max),
                             sr=sr, x_axis='time', y_axis='log', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Spectrogram - {file_name}")
    plt.tight_layout()
    output_path = os.path.join(output_folder, f"spectrogram_{file_name}.png")
    plt.savefig(output_path)
    plt.close()

# Procesar los archivos
file_list = [f for f in os.listdir(input_folder) if f.endswith('.wav')]

for file_name in file_list:
    print(f"Procesando: {file_name}")
    file_path = os.path.join(input_folder, file_name)

    # Seleccionar un archivo aleatorio de la misma carpeta para combinar
    file_name_random = np.random.choice([f for f in file_list if f != file_name])
    random_file_path = os.path.join(input_folder, file_name_random)

    print(f"  -> Combinando con: {file_name_random}")

    # Cargar ambos audios
    audio1, sr1 = librosa.load(file_path, sr=None)
    audio2, sr2 = librosa.load(random_file_path, sr=None)

    # Asegurar que las tasas de muestreo sean iguales
    if sr1 != sr2:
        raise ValueError("Los archivos de audio tienen diferentes tasas de muestreo.")

    # Generar valores aleatorios para los parámetros de EMDA
    alpha = round(np.random.uniform(0.3, 0.7), 2)  # Ponderación entre 0.3 y 0.7
    beta = round(np.random.uniform(0, 1), 2)  # Peso relativo
    delay = round(np.random.uniform(0, 0.05), 3)  # Retraso en segundos (0 a 50 ms)

    print(f"  -> Parámetros EMDA: alpha={alpha}, beta={beta}, delay={delay}s")

    # Aplicar EMDA Augmentation
    combined_spec = apply_emda(audio1, audio2, alpha, beta, delay, sr1)

    # Guardar el espectrograma combinado
    output_name = f"emda_alpha{alpha}_beta{beta}_delay{delay}s_{file_name}_{file_name_random}".replace('.wav', '')
    save_spectrogram(combined_spec, sr1, output_name)

print("Espectrogramas generados con Spectrogram EMDA Augmentation.")

Procesando: VNE_90_P2_17_VM.wav
  -> Combinando con: VNE_103_P1_88_MH.wav
  -> Parámetros EMDA: alpha=0.62, beta=0.48, delay=0.021s
Procesando: VNE_103_P1_88_MH.wav
  -> Combinando con: VNE_102_P2_79_TM.wav
  -> Parámetros EMDA: alpha=0.37, beta=0.68, delay=0.028s
Procesando: VNE_92_P2_18_SI.wav
  -> Combinando con: VNE_103_P1_88_MH.wav
  -> Parámetros EMDA: alpha=0.66, beta=0.99, delay=0.044s
Procesando: VNE_102_P2_79_TM.wav
  -> Combinando con: VNE_92_P2_18_SI.wav
  -> Parámetros EMDA: alpha=0.57, beta=0.09, delay=0.019s
Procesando: VNE_31_P2_42_BI.wav
  -> Combinando con: VNE_92_P2_18_SI.wav
  -> Parámetros EMDA: alpha=0.58, beta=0.44, delay=0.046s
Procesando: VNE_104_P1_57_GE.wav
  -> Combinando con: VNE_90_P2_17_VM.wav
  -> Parámetros EMDA: alpha=0.51, beta=0.4, delay=0.006s
Espectrogramas generados con Spectrogram EMDA Augmentation.


**FFT**

In [7]:
# Código para generar la Transformada de Fourier (FFT) de los espectrogramas generados con EMDA

# Instalar las dependencias
!pip install librosa matplotlib

# Importar librerías
import librosa
import numpy as np
import os
import matplotlib.pyplot as plt


# Configuración de rutas
input_folder = '/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/RESULTS/SpectrogramAugmentation/EMDAAugmenter/SPECTROGRAMS'  # Carpeta con los espectrogramas generados
output_folder = '/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/RESULTS/SpectrogramAugmentation/EMDAAugmenter/FFT'  # Carpeta para guardar las gráficas FFT
os.makedirs(output_folder, exist_ok=True)

# Función para calcular y guardar la FFT de un espectrograma
def plot_fft(spectrogram, sr, file_name):
    # Calcular la magnitud promedio a lo largo del tiempo
    magnitude = np.mean(np.abs(spectrogram), axis=1)
    frequency = np.linspace(0, sr // 2, len(magnitude))

    # Generar la gráfica
    plt.figure(figsize=(10, 6))
    plt.plot(frequency, magnitude, alpha=0.7)
    plt.title(f"FFT - {file_name}")
    plt.xlabel("Frecuencia (Hz)")
    plt.ylabel("Magnitud")
    plt.grid()

    # Guardar la gráfica
    output_path = os.path.join(output_folder, f"fft_{file_name}.png")
    plt.savefig(output_path)
    plt.close()

# Procesar los archivos de espectrogramas generados
for file_name in os.listdir(input_folder):
    if file_name.endswith('.png'):
        print(f"Procesando FFT de espectrograma: {file_name}")
        file_path = os.path.join(input_folder, file_name)

        # Cargar el espectrograma
        spectrogram = librosa.amplitude_to_db(np.abs(librosa.stft(librosa.load(file_path, sr=None)[0])), ref=np.max)

        # Calcular y guardar la FFT
        plot_fft(spectrogram, sr=22050, file_name=file_name.replace('.png', ''))

print("Gráficas FFT generadas para los espectrogramas (EMDA).")


Procesando FFT de espectrograma: spectrogram_emda_alpha0.62_beta0.48_delay0.021s_VNE_90_P2_17_VM_VNE_103_P1_88_MH.png


<ipython-input-7-81f333d749f7>:44: UserWarning: PySoundFile failed. Trying audioread instead.
  spectrogram = librosa.amplitude_to_db(np.abs(librosa.stft(librosa.load(file_path, sr=None)[0])), ref=np.max)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


NoBackendError: 